In [1]:
import pandas as pd

In [2]:
train=pd.read_csv("Data/Train.csv")
test=pd.read_csv("Data/Test.csv")
waste=pd.read_csv("Data/waste_management.csv")
water=pd.read_csv("Data/water_sources.csv")
toilet=pd.read_csv("Data/toilets.csv")

In [3]:
waste.head()

,Year,Month,10u,10v,2d,2t,evabs,evaow,evatc,evavt,...,tp,swvl1,swvl2,swvl3,swvl4,Transformed_Latitude,Transformed_Longitude,Month_Year,lat_lon,Month_Year_lat_lon
0,2019,1,-1.006850,-0.714544,293.502,297.417,-0.002489,-0.001031,-0.000989,-3.445890e-08,...,0.006655,0.346542,0.335724,0.264359,0.300018,-8.58518,68.25058,1_2019,-8.58518_68.25058,1_2019_-8.58518_68.25058
1,2019,2,-1.408560,-0.662555,292.943,297.978,-0.002556,-0.001076,-0.000734,-3.818420e-08,...,0.003201,0.254517,0.259918,0.281769,0.296188,-8.58518,68.25058,2_2019,-8.58518_68.25058,2_2019_-8.58518_68.25058
2,2019,3,-0.892110,-0.449448,293.250,297.989,-0.002535,-0.000980,-0.000713,-3.678720e-08,...,0.004682,0.278152,0.284195,0.261597,0.291672,-8.58518,68.25058,3_2019,-8.58518_68.25058,3_2019_-8.58518_68.25058
3,2019,4,-0.392348,0.384925,293.934,296.542,-0.001748,-0.000658,-0.001343,-2.700840e-08,...,0.007518,0.385513,0.378616,0.309784,0.288925,-8.58518,68.25058,4_2019,-8.58518_68.25058,4_2019_-8.58518_68.25058
4,2019,5,-0.447104,0.581028,292.571,294.726,-0.001374,-0.000462,-0.001139,-2.048910e-08,...,0.005989,0.397659,0.400055,0.392654,0.309311,-8.58518,68.25058,5_2019,-8.58518_68.25058,5_2019_-8.58518_68.25058


In [15]:
train.dtypes

ID                                object
Total                            float64
Location                          object
Category_Health_Facility_UUID     object
Disease                           object
Month                              int64
Year                               int64
Transformed_Latitude             float64
Transformed_Longitude            float64
dtype: object

In [4]:
train.head()

,ID,Total,Location,Category_Health_Facility_UUID,Disease,Month,Year,Transformed_Latitude,Transformed_Longitude
0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Dysentery,12,2022,-8.62982,68.23267
1,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Typhoid,12,2022,-8.62982,68.23267
2,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267
3,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267
4,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267


In [5]:
print("Train Null Values",train.isna().sum().max())
print("Waste Null Values",waste.isna().sum().max())
print("Water Null Values",water.isna().sum().max())
print("Toilet Null Values",toilet.isna().sum().max())

Train Null Values 1
Waste Null Values 0
Water Null Values 60
Toilet Null Values 0


In [6]:
train.dropna(inplace=True)
water.dropna(inplace=True)

In [7]:
print("Train Null Values",train.isna().sum().max())
print("Waste Null Values",waste.isna().sum().max())
print("Water Null Values",water.isna().sum().max())
print("Toilet Null Values",toilet.isna().sum().max())

Train Null Values 0
Waste Null Values 0
Water Null Values 0
Toilet Null Values 0


In [8]:
waste.rename(columns=lambda col: f"waste_{col}",inplace=True)
water.rename(columns=lambda col: f"water_{col}",inplace=True)
toilet.rename(columns=lambda col: f"toilet_{col}",inplace=True)

In [9]:
from scipy.spatial import cKDTree

def find_nearest(hospital_df, location_df, lat_col, lon_col, id_col):
    
    # Create a cKDTree for efficient nearest neighbour search
    tree = cKDTree(location_df[[lat_col, lon_col]].values)
    nearest = {}
    
    # Loop through each hospital and find the nearest site in location_df
    for _, row in hospital_df.iterrows():
        dist, idx = tree.query([row['Transformed_Latitude'], row['Transformed_Longitude']],k=1)
        nearest[row['ID']] = location_df.iloc[idx][id_col]
    return nearest

In [10]:
merged_data=train.copy()
datasets = [
    (toilet, 'toilet', 'toilet_Month_Year_lat_lon'),
    (waste, 'waste', 'waste_Month_Year_lat_lon'),
    (water, 'water', 'water_Month_Year_lat_lon'),
]

In [11]:
for df, prefix, id_col in datasets:
    nearest = find_nearest(merged_data, df, f"{prefix}_Transformed_Latitude", f"{prefix}_Transformed_Longitude", id_col)
    nearest_df = pd.DataFrame(list(nearest.items()), columns=['ID', id_col])
    merged_data = merged_data.merge(nearest_df, on="ID").merge(df, on=id_col)

In [12]:
merged_data.shape

(24375, 138)

In [13]:
merged_data.columns

Index(['ID', 'Total', 'Location', 'Category_Health_Facility_UUID', 'Disease',
       'Month', 'Year', 'Transformed_Latitude', 'Transformed_Longitude',
       'toilet_Month_Year_lat_lon',
       ...
       'water_e', 'water_tp', 'water_swvl1', 'water_swvl2', 'water_swvl3',
       'water_swvl4', 'water_Transformed_Latitude',
       'water_Transformed_Longitude', 'water_Month_Year', 'water_lat_lon'],
      dtype='object', length=138)

In [14]:
merged_data.head()

,ID,Total,Location,Category_Health_Facility_UUID,Disease,Month,Year,Transformed_Latitude,Transformed_Longitude,toilet_Month_Year_lat_lon,...,water_e,water_tp,water_swvl1,water_swvl2,water_swvl3,water_swvl4,water_Transformed_Latitude,water_Transformed_Longitude,water_Month_Year,water_lat_lon
0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Dysentery,12,2022,-8.62982,68.23267,4_2020_-8.62979_68.23582,...,-0.002589,0.002667,0.325867,0.325302,0.281708,0.24585,-8.62966,68.23589,12_2023,-8.62966_68.23589
1,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Typhoid,12,2022,-8.62982,68.23267,4_2020_-8.62979_68.23582,...,-0.002589,0.002667,0.325867,0.325302,0.281708,0.24585,-8.62966,68.23589,12_2023,-8.62966_68.23589
2,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267,4_2020_-8.62979_68.23582,...,-0.002589,0.002667,0.325867,0.325302,0.281708,0.24585,-8.62966,68.23589,12_2023,-8.62966_68.23589
3,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267,4_2020_-8.62979_68.23582,...,-0.002589,0.002667,0.325867,0.325302,0.281708,0.24585,-8.62966,68.23589,12_2023,-8.62966_68.23589
4,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267,4_2020_-8.62979_68.23582,...,-0.002589,0.002667,0.325867,0.325302,0.281708,0.24585,-8.62966,68.23589,12_2023,-8.62966_68.23589


In [17]:
merged_data["Disease"].value_counts()

Disease
Malaria             8863
Diarrhea            6644
Typhoid             2218
Schistosomiasis     2217
Intestinal Worms    2216
Dysentery           2191
Cholera               26
Name: count, dtype: int64